<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h1 align="center" style="color:#ffffff;"><b>Future Sales Prediction 📈💹</b></h1>
</div>

<div style="background-color:#000000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <img src="https://media.tenor.com/mg9uZjeNVncAAAAC/sales-imp-dance.gif" alt="Animated GIF">
</div>

In [1]:
!pip3 install -q xgboost lightgbm

<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>IMPORT DEPENDENCIES</b></h2>
</div>

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
item_cats = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
item_cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
sales_test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
sales_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
sample_submission = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>DATA PRE-PROCESSING AND EDA</b></h2>
</div>

In [9]:
items_merged = pd.merge(items,item_cats, on='item_category_id', how='left')
items_merged.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD


In [10]:
new_train = pd.merge(sales_train,items_merged, on='item_id', how='left')
new_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [11]:
new_train.isnull().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
dtype: int64

In [12]:
new_train = pd.merge(new_train,shops, on='shop_id', how='left')
new_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [13]:
new_train.isnull().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
dtype: int64

In [14]:
new_train.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name'],
      dtype='object')

In [15]:
sample_submission.columns

Index(['ID', 'item_cnt_month'], dtype='object')

In [16]:
sales_test.columns

Index(['ID', 'shop_id', 'item_id'], dtype='object')

In [17]:
new_train['new_date'] = pd.to_datetime(new_train['date'])
# Extract year and month from the 'date' column
new_train['year']  = new_train['new_date'].dt.year
new_train['month'] = new_train['new_date'].dt.month

In [18]:
new_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name,new_date,year,month
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013-02-01,2013,2
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум""",2013-03-01,2013,3
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум""",2013-05-01,2013,5
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум""",2013-06-01,2013,6
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум""",2013-01-15,2013,1


In [19]:
new_train.item_cnt_day[new_train.item_cnt_day<=0]

2         -1.0
148       -1.0
175       -1.0
807       -1.0
1041      -1.0
          ... 
2934243   -1.0
2934462   -1.0
2935263   -1.0
2935643   -1.0
2935779   -1.0
Name: item_cnt_day, Length: 7356, dtype: float64

In [20]:
# Group by year, month, and item_id and aggregate the item_cnt_day and item_price columns
monthly_sales = new_train.groupby(['shop_id','item_id','year', 'month','new_date']).agg({'item_cnt_day': 'sum', 'item_price': 'sum'}).reset_index()

# Rename the aggregated columns
monthly_sales = monthly_sales.rename(columns={'item_cnt_day': 'monthly_sold_number', 'item_price': 'monthly_sold_price'})

# Display the resulting DataFrame
monthly_sales.head()

,shop_id,item_id,year,month,new_date,monthly_sold_number,monthly_sold_price
0,0,30,2013,2,2013-02-15,2.0,265.0
1,0,30,2013,2,2013-02-16,9.0,265.0
2,0,30,2013,2,2013-02-17,4.0,265.0
3,0,30,2013,2,2013-02-18,4.0,265.0
4,0,30,2013,2,2013-02-20,2.0,265.0


In [21]:
train_data = monthly_sales.rename(columns={
    'new_date':'date',
    'shop_id':'shop_id',
    'item_id':'item_id',
    'year':'year',
    'month':'month',
    'monthly_sold_number':'sold_number',
    'monthly_sold_price' :'sold_price'
});
train_data.head()

,shop_id,item_id,year,month,date,sold_number,sold_price
0,0,30,2013,2,2013-02-15,2.0,265.0
1,0,30,2013,2,2013-02-16,9.0,265.0
2,0,30,2013,2,2013-02-17,4.0,265.0
3,0,30,2013,2,2013-02-18,4.0,265.0
4,0,30,2013,2,2013-02-20,2.0,265.0


In [22]:
train_data.isnull().sum()

shop_id        0
item_id        0
year           0
month          0
date           0
sold_number    0
sold_price     0
dtype: int64

In [23]:
train_data.duplicated().sum()

0

In [24]:
print(f"Train Shape : {train_data.shape}")

Train Shape : (2935821, 7)


In [25]:
train_data

,shop_id,item_id,year,month,date,sold_number,sold_price
0,0,30,2013,2,2013-02-15,2.0,265.0
1,0,30,2013,2,2013-02-16,9.0,265.0
2,0,30,2013,2,2013-02-17,4.0,265.0
3,0,30,2013,2,2013-02-18,4.0,265.0
4,0,30,2013,2,2013-02-20,2.0,265.0
...,...,...,...,...,...,...,...
2935816,59,22164,2015,11,2015-11-02,1.0,749.0
2935817,59,22167,2013,3,2013-03-12,1.0,299.0
2935818,59,22167,2013,10,2013-10-25,1.0,299.0
2935819,59,22167,2013,12,2013-12-14,1.0,299.0


In [26]:
train_data['ID'] = train_data.groupby(['shop_id', 'item_id', 'year', 'month']).ngroup()

In [27]:
train_data.head()

,shop_id,item_id,year,month,date,sold_number,sold_price,ID
0,0,30,2013,2,2013-02-15,2.0,265.0,0
1,0,30,2013,2,2013-02-16,9.0,265.0,0
2,0,30,2013,2,2013-02-17,4.0,265.0,0
3,0,30,2013,2,2013-02-18,4.0,265.0,0
4,0,30,2013,2,2013-02-20,2.0,265.0,0


In [28]:
X = train_data.drop(['sold_number','sold_price','date','year','month'], axis=1)
y = train_data['sold_number']

#ID	shop_id	item_id
# Split the data into training and testing sets
x_train,x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# Print the shapes of the resulting sets
print("X_train shape :", x_train.shape)
print("X_test shape  :", x_test.shape)
print("y_train shape :", y_train.shape)
print("y_test shape  :", y_test.shape)

X_train shape : (2055074, 3)
X_test shape  : (880747, 3)
y_train shape : (2055074,)
y_test shape  : (880747,)


<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>MODEL ENGINEERING</b></h2>
</div>

In [29]:
# XGBoost
xgb_model = xgb.XGBRegressor()
xgb_model.fit(x_train, y_train)

# LightGBM
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(x_train, y_train)

LGBMRegressor()

In [30]:
# XGBoost
xgb_predictions = xgb_model.predict(x_test)

# LightGBM
lgb_predictions = lgb_model.predict(x_test)

In [31]:
# Calculate MSE
xgb_mse = mean_squared_error(y_test, xgb_predictions)
lgb_mse = mean_squared_error(y_test, lgb_predictions)

# Calculate MAE
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
lgb_mae = mean_absolute_error(y_test, lgb_predictions)

# Calculate RMSE
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
lgb_rmse = mean_squared_error(y_test, lgb_predictions, squared=False)

# Calculate R-squared score
xgb_r2 = r2_score(y_test, xgb_predictions)
lgb_r2 = r2_score(y_test, lgb_predictions)

print("--------------------: XGBoost Metrics :----------------------")
print("MSE       : ", xgb_mse)
print("MAE       : ", xgb_mae)
print("RMSE      : ", xgb_rmse)
print("R-squared : ", xgb_r2)
print()

print("-------------------: LightGBM Metrics :----------------------")
print("MSE       : ", lgb_mse)
print("MAE       : ", lgb_mae)
print("RMSE      : ", lgb_rmse)
print("R-squared : ", lgb_r2)

--------------------: XGBoost Metrics :----------------------
MSE       :  4.0863106873638655
MAE       :  0.30235208161165816
RMSE      :  2.0214625119857814
R-squared :  0.024129107268091254

-------------------: LightGBM Metrics :----------------------
MSE       :  4.096628184497287
MAE       :  0.30279772072660605
RMSE      :  2.024012891386141
R-squared :  0.021665137710053073


In [32]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

# Calculate MAPE
xgb_mape = mean_absolute_percentage_error(y_test, xgb_predictions)
lgb_mape = mean_absolute_percentage_error(y_test, lgb_predictions)

# Calculate SMAPE
xgb_smape = symmetric_mean_absolute_percentage_error(y_test, xgb_predictions)
lgb_smape = symmetric_mean_absolute_percentage_error(y_test, lgb_predictions)

print("XGBoost Metrics:")
print("MAPE:", xgb_mape)
print("SMAPE:", xgb_smape)
print()

print("LightGBM Metrics:")
print("MAPE:", lgb_mape)
print("SMAPE:", lgb_smape)


XGBoost Metrics:
MAPE: 16.292159297040815
SMAPE: 16.25336562878239

LightGBM Metrics:
MAPE: 16.142606658592644
SMAPE: 16.588737438660075


In [33]:
sales_test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [34]:
xgb_model.predict(sales_test)

array([1.2459811, 0.9930698, 1.2374105, ..., 0.1983811, 0.9584357,
       1.0123454], dtype=float32)

In [35]:
lgb_model.predict(sales_test)

array([1.1573438 , 1.06214954, 1.06214954, ..., 1.12574359, 1.12574359,
       1.12574359])

In [36]:
# Create a DataFrame for submission
submission_lgb_model = pd.DataFrame({'ID': sales_test.ID, 'item_cnt_month': lgb_model.predict(sales_test)})
# Save the DataFrame to a CSV file
submission_lgb_model.to_csv('submission_lgb_model.csv', index=False)

In [37]:
# Create a DataFrame for submission
submission_xgb_model = pd.DataFrame({'ID': sales_test.ID, 'item_cnt_month': xgb_model.predict(sales_test)})
# Save the DataFrame to a CSV file
submission_xgb_model.to_csv('submission_xgb_model.csv', index=False)